# import data

In [ ]:
import os 
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [ ]:
data_path = '/Users/bokie/code/python/CMS/可转债套利/cms/stat_arb/stk_data'
df_price_dic = {}
df_return_dic = {}
industry = os.listdir(data_path+'/price')

for tmp_industry in industry:
    tmp_price_path = data_path + '/price/' + tmp_industry
    tmp_return_path = data_path + '/return/' + tmp_industry
    tmp_df_price = pd.read_pickle(tmp_price_path)
    tmp_df_return = pd.read_pickle(tmp_return_path)
    tmp_key = tmp_industry.split('.')[0]
    df_price_dic[tmp_key] = tmp_df_price
    df_return_dic[tmp_key] = tmp_df_return

In [ ]:
corr_dic = {}
for k,v in df_return_dic.items():
    corr_mat = v.corr(method='pearson')
    for i in range(len(corr_mat)):
        corr_mat.iloc[i,i] = np.nan
    corr_dic[k] = corr_mat

# pair correlation

In [ ]:
corr_pair_dic = {}
for tmp_industry in corr_dic.keys():
    tmp_corr_max = corr_dic[tmp_industry].max()
    tmp_df_corr = corr_dic[tmp_industry]

    tmp_corr_max,type(tmp_corr_max),tmp_corr_max.index.tolist()
    industry_corr_pair_dic = {}

    main_code_list = tmp_corr_max.index.tolist()
    for main_code in main_code_list:
        tmp_line = tmp_df_corr.loc[main_code]
        tmp_line = pd.DataFrame(tmp_line)
        for index,row in tmp_line.iterrows():
            if row[main_code] == tmp_corr_max.loc[main_code]:
                # print(index)
                sec_code = index
                tmp_corr = tmp_corr_max.loc[main_code]
        # print(f'\'{main_code}\',\'{sec_code}\'','@',tmp_corr)
        tmp_key = main_code+'@'+sec_code
        tmp_key_reverse = sec_code+'@'+main_code
        if (tmp_key not in industry_corr_pair_dic) and (tmp_key_reverse not in industry_corr_pair_dic):
            industry_corr_pair_dic[tmp_key] = tmp_corr
        
    corr_pair_dic[tmp_industry] = sorted(industry_corr_pair_dic.items(), key=lambda x:x[1],reverse=True)


In [ ]:
df_corr_pair = pd.DataFrame(columns=['industry','corr'])
for k,v in corr_pair_dic.items():
    for pair_corr in v:
        tmp_pair = pair_corr[0]
        tmp_corr = pair_corr[1]
        if tmp_corr>0:
            # print(k,tmp_pair,tmp_corr)
            df_corr_pair.loc[tmp_pair] = [k,tmp_corr]
            # print("%-10s\t\t%-10s\t\t%-10s"%(k,tmp_pair,tmp_corr))
df_corr_pair = df_corr_pair.reset_index()
df_corr_pair.columns = ['pair','industry','corr']
df_corr_pair

# pair cointegration test

In [ ]:
df_return_dic['煤炭'][['600395.SH','002128.SZ']]

In [ ]:
def cointegration_test(y, x):
    ols_result = sm.OLS(y, x).fit()
    return adfuller(ols_result.resid)

df_coint_pair = pd.DataFrame(columns=['industry','coint','reverse coint'])
for index,row in df_corr_pair.iterrows():
    pair = row['pair']
    industry = row['industry']
    pair_return = df_return_dic[industry]
    x_code = pair.split('@')[0]
    y_code = pair.split('@')[1]
    x = pair_return[x_code].tolist()
    y = pair_return[y_code].tolist()
    res = cointegration_test(y, x),
    reverse_res = cointegration_test(x, y)
    
    df_coint_pair.loc[pair] = [industry,res,reverse_res]
    print(index,industry,pair)

In [ ]:
df_coint_pair

In [ ]:
df_coint_pair.to_excel('coint_res.xlsx')